In [ ]:
#Necessary Library
import numpy as np
import os
import matplotlib.pyplot as plt

import cv2
from tqdm import tqdm
import io
import seaborn as sns

# Shuffle arrays or sparse matrices in a consistent way
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import EfficientNetB0, EfficientNetB7
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard,ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
from keras.utils.vis_utils import plot_model

import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output

#Extra functionality for TensorFlow
#conda install -c esri tensorflow-addons
import tensorflow_addons as tfa

from datetime import datetime
from keras import regularizers

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

print("Importing Reference done..")

In [ ]:
#Variable declaration
BASE_DIR = "/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/"
SAVE_DIR = "/kaggle/working/"
#SAVE_DIR = BASE_DIR + "model_bilateral_smote_100_RMSprop_tr_001_141022_final/"

#Check the saved directory and create if not found
isExist = os.path.exists(SAVE_DIR)
if not isExist:
   os.makedirs(SAVE_DIR)

In [ ]:
labels = ['CONTROL', 'AD', 'PD']
X_train = [] #Training Dataset
Y_train = [] #Training Labels

X_train_img = [] #Training Dataset
Y_train_img = [] #Training Labels

X_Test_AD = []
Y_Test_AD = []
X_Test_PD = []
Y_Test_PD = []
X_Test_CONTROL = []
Y_Test_CONTROL = []

image_size=224
image_size_X=224
image_size_Y=224

#0=resize only, 1=CLAHE, 2=denoise, 3=dnoise_after_clahe, 4=old_preprocess
preprocess_method=2

CLASSES = ['CONTROL', 'AD', 'PD']
model_name_to_save = 'hybrid_efficientnet.h5'

epochs_size = 50
batch_size_here = 32

In [ ]:
## Plot preprocessed image
import cv2
import argparse
import numpy as np

#This function will plot two images side by side
def plot_image(image_1, image_2, title_1="Orignal", title_2="Preprocessed"):
    
    print('Original img shape: ' + str(image_1.shape))
    #print(type(image_1.shape))    
    print('Processed img shape: ' + str(image_2.shape))
    #print(type(image_2.shape))
    
    #8/8 inch figure
    width_inch = 8
    height_inch = 8
    plt.figure(figsize=(width_inch, height_inch))
    row = 1
    column = 2
    
    position = 1
    plt.subplot(row, column, position)
    plt.imshow(image_1)
    plt.title(title_1)
    
    position = 2
    plt.subplot(row, column, position)
    plt.imshow(image_2)
    plt.title(title_2)
            
    #save the image
    filter_img_name = title_1
    filter_img_name = SAVE_DIR + filter_img_name.replace(': Original', '')
    print(filter_img_name)
    plt.savefig(filter_img_name, dpi=100)
    
    #show the image
    plt.show()

In [ ]:
#Plot preprocessed image
#This function will plot three images side by side
def plot_3_images(image_1, image_2, image_3, img_type):
    
    print('Original img shape: ' + str(image_1.shape))
    #print(type(image_1.shape))    
    print('Processed img shape: ' + str(image_3.shape))
    #print(type(image_2.shape))
    
    #12/8 inch figure
    width_inch = 12
    height_inch = 8
    plt.figure(figsize=(width_inch, height_inch))
    row = 1
    column = 3
    
    position = 1
    plt.subplot(row, column, position)
    plt.imshow(image_1)
    plt.title(img_type + "_Original")
    
    position = 2
    plt.subplot(row, column, position)
    plt.imshow(image_2)
    plt.title(img_type + "_after CLAHE")
            
    position = 3
    plt.subplot(row, column, position)
    plt.imshow(image_3)
    plt.title(img_type + "_after Denoise")
    
    #save the image
    filter_img_name = img_type + ".png"
    filter_img_name = SAVE_DIR + filter_img_name
    print(filter_img_name)
    plt.savefig(filter_img_name, dpi=100)
    
    #show the image
    plt.show()

In [ ]:
#funtion for mean filter/blur
def resize_only(folderPath, image_name):
    
    image = cv2.imread(os.path.join(folderPath, image_name))
    
    #preprocessing
    processed_image = cv2.resize(image, (image_size_X, image_size_Y))  
    
    #Images to plot
    img_list = ['PD_2.png', 'CONTROL_PD_1.png', 'AD_9.png']
    #print(image_name)
    if image_name in img_list:
        print("resize_only: " + folderPath + image_name)
        plot_image(image, processed_image, title_1 = image_name + ": Original", title_2 = image_name + ": Processed") 
        
    return processed_image
    

In [ ]:
#Preprocessing defination
#https://www.kaggle.com/code/natigmamishov/complete-guide-to-image-processing-with-opencv
#funtion for mean filter/blur
def preprocess_image(folderPath, image_name):
    
    image = cv2.imread(os.path.join(folderPath, image_name))
    
    #preprocessing
    processed_image = cv2.resize(image, (image_size_X, image_size_Y))        
    processed_image = cv2.cvtColor(np.array(processed_image), cv2.COLOR_RGB2BGR)
    processed_image = cv2.bilateralFilter(processed_image, 15, 75, 75)
    
    # denoising the image using the cv2.fastNlMeansDenoising() function  
    processed_image = cv2.fastNlMeansDenoising(image, None, 15, 7, 21 )  
        
    #image = cv2.imread(img)
    #processed_image = cv2.blur(image, kernel)    
    #cv2.filter2D(image,-1,kernel) = cv2.filter2D(image,-1,kernel)
    #writing_path = PRE_PRO_DIR + target_folder + '/' + image_name;
    #print(writing_path)
    #cv2.imwrite(writing_path, processed_image)
    #plot_image(image, processed_image)
    
    #Images to plot
    img_list = ['PD_2.png', 'CONTROL_PD_1.png', 'AD_9.png']
    #print(image_name)
    if image_name in img_list:
        print("preprocess_image: " + folderPath + image_name)
        plot_image(image, processed_image, title_1 = image_name + ": Original", title_2 = image_name + ": Processed") 
    
    return processed_image
    


In [ ]:
#CLAHE Histogram Equalization
#Contrast Limited Adaptive Histogram Equalization (CLAHE)
#https://www.geeksforgeeks.org/clahe-histogram-eqalization-opencv/
def preprocess_CLAHE(folderPath, image_name):
    
    # Open the image as a grayscale image
    #image = cv2.imread(os.path.join(folderPath, image_name), 0 )
    
    # Open the image as original
    image = cv2.imread(os.path.join(folderPath, image_name))
    
    #Convert to grayscale if open as original
    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #changes the shape to (X, Y)
    
    processed_image = cv2.resize(processed_image, (image_size_X, image_size_Y)) 
    
    # The initial processing of the image
    # image = cv2.medianBlur(image, 3)
    
    # The declaration of CLAHE
    # clipLimit -> Threshold for contrast limiting
    
    #clahe = cv2.createCLAHE(clipLimit = 5)    
    #processed_image = clahe.apply(processed_image) + 30
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    processed_image = clahe.apply(processed_image)
    
    # Convert the image back to RGB
    #processed_image = cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR)
    processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)
    
    #changes the shape to (150, 150, 3)
    
    #Images to plot
    img_list = ['PD_2.png', 'CONTROL_PD_1.png', 'AD_9.png']
    #print(image_name)
    if image_name in img_list:
        print("preprocess_CLAHE: " + folderPath + image_name)
        plot_image(image, processed_image, title_1 = image_name + ": Original", title_2 = image_name + ": Processed") 
    
    return processed_image

In [ ]:
def image_denoise(folderPath, image_name):
    
    # Open the image as a grayscale image
    #image = cv2.imread(os.path.join(folderPath, image_name), 0 )
    
    # Open the image as original
    image = cv2.imread(os.path.join(folderPath, image_name))
    #Convert to grayscale
    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Record the dimensions of the image
    #height, width = image.shape
    
    # We enlarge the image to the following dimensions: 200 pixels in height
    # Note that we resize images while maintaining the aspect ratio.
    processed_image = cv2.resize(processed_image, (image_size_X, image_size_Y))  
    
    
    #processed_image = cv2.fastNlMeansDenoising(processed_image, None, 23, 7 ,21)
    #makes the image full black
    #processed_image = cv2.threshold(processed_image, 128, 255, cv2.THRESH_BINARY_INV)[1]
    
    processed_image = cv2.fastNlMeansDenoising(processed_image, None, 10, 7, 21)
    #processed_image = cv2.adaptiveThreshold(processed_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    #resize to x, y, z
    processed_image = cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR)
    
    # Perform binarization of the image (covert to black and white)
    #ret, processed_image = cv2.threshold(processed_image,127,255,cv2.THRESH_BINARY)
    
    # Save and replace the old image with the new image
    #cv2.imwrite( imageName, img)
    
    # Now sharpen the image and improve resolution with PIL 
    #im = Image.open( imageName )
    
    # Save the image with higher resolution
    #im.save(imageName, dpi=(600,600))
    
    # Wait for image to save
    #time.sleep(10)
    
    #Images to plot
    img_list = ['PD_2.png', 'CONTROL_PD_1.png', 'AD_9.png']
    #print(image_name)
    if image_name in img_list:
        print("image_denoise: " + folderPath + image_name)
        plot_image(image, processed_image, title_1 = image_name + ": Original", title_2 = image_name + ": Processed") 
    
    return processed_image

In [ ]:
def image_denoise_after_CLAHE(folderPath, image_name, clahe_image):
    
    # Open the image as a grayscale image
    #image = cv2.imread(os.path.join(folderPath, image_name), 0 )
    
    # Open the image as original
    image = cv2.imread(os.path.join(folderPath, image_name))
    
    #Convert to grayscale
    processed_image = cv2.cvtColor(clahe_image, cv2.COLOR_BGR2GRAY)

    # Record the dimensions of the image
    #height, width = image.shape
    
    # We enlarge the image to the following dimensions: 200 pixels in height
    # Note that we resize images while maintaining the aspect ratio.
    processed_image = cv2.resize(processed_image, (image_size_X, image_size_Y))  
    
    
    #processed_image = cv2.fastNlMeansDenoising(processed_image, None, 23, 7 ,21)
    #makes the image full black
    #processed_image = cv2.threshold(processed_image, 128, 255, cv2.THRESH_BINARY_INV)[1]
    
    processed_image = cv2.fastNlMeansDenoising(processed_image, None, 10, 7, 21)
    #processed_image = cv2.adaptiveThreshold(processed_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    #resize to x, y, z
    #processed_image = cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR)
    processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)
    
    # Perform binarization of the image (covert to black and white)
    #ret, processed_image = cv2.threshold(processed_image,127,255,cv2.THRESH_BINARY)
    
    # Save and replace the old image with the new image
    #cv2.imwrite( imageName, img)
    
    # Now sharpen the image and improve resolution with PIL 
    #im = Image.open( imageName )
    
    # Save the image with higher resolution
    #im.save(imageName, dpi=(600,600))
    
    # Wait for image to save
    #time.sleep(10)
    
    #Images to plot
    img_list = ['PD_2.png', 'CONTROL_PD_1.png', 'AD_9.png']
    #print(image_name)
    if image_name == 'PD_2.png':
        print("image_denoise_after_CLAHE: " + folderPath + image_name)
        plot_3_images(image, clahe_image, processed_image, "PD") 
    elif image_name == 'CONTROL_PD_1.png':
        plot_3_images(image, clahe_image, processed_image, "CONTROL") 
    elif image_name == 'AD_9.png':
        plot_3_images(image, clahe_image, processed_image, "AD") 
    
    return processed_image

In [ ]:
#Image preprocessed and appended with Label to list
print('preprocess method: ' + str(preprocess_method))
for label in labels:
    folderPath = os.path.join(BASE_DIR, 'train', label)
    for img_name in tqdm(os.listdir(folderPath)):
        #image = cv2.imread(os.path.join(folderPath, j))
        #image = cv2.resize(image, (image_size_X, image_size_Y))        
        #images = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        #image = cv2.bilateralFilter(image, 15, 75, 75)
        
        #preprocessing
        #without preprocessing
        if preprocess_method == 0:
            image = resize_only(folderPath, img_name)
        
        #CLAHE on original image
        elif preprocess_method == 1:        
            image = preprocess_CLAHE(folderPath, img_name)
        
        #denoise on original image
        elif preprocess_method == 2:        
            image = image_denoise(folderPath, img_name)
            
        #denoise after CLAHE
        elif preprocess_method == 3:        
            image = preprocess_CLAHE(folderPath, img_name)
            image = image_denoise_after_CLAHE(folderPath, img_name, image)
        
        #without preprocessing
        elif preprocess_method == 4:
            image = preprocess_image(folderPath, img_name)
        
        X_train_img.append(image)
        Y_train_img.append(label)
        
for label in labels:
    # Join two or more pathname components
    folderPath = os.path.join(BASE_DIR, 'test', label) 
    for img_name in tqdm(os.listdir(folderPath)):
        #image = cv2.imread(os.path.join(folderPath, j))
        #image = cv2.resize(image, (image_size_X, image_size_Y))        
        #images = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        #image = cv2.bilateralFilter(image, 15, 75, 75)
        
        #preprocessing
        #without preprocessing
        if preprocess_method == 0:
            image = resize_only(folderPath, img_name)
        
        #CLAHE on original image
        elif preprocess_method == 1:        
            image = preprocess_CLAHE(folderPath, img_name)
        
        #denoise on original image
        elif preprocess_method == 2:        
            image = image_denoise(folderPath, img_name)
            
        #denoise after CLAHE
        elif preprocess_method == 3:        
            image = preprocess_CLAHE(folderPath, img_name)
            image = image_denoise_after_CLAHE(folderPath, img_name, image)
        
        #without preprocessing
        elif preprocess_method == 4:
            image = preprocess_image(folderPath, img_name)
        
        X_train_img.append(image)
        Y_train_img.append(label)
        
        if label == 'AD':
            X_Test_AD.append(image)
            Y_Test_AD.append(label)
        elif label == 'PD':
            X_Test_PD.append(image)
            Y_Test_PD.append(label)
        elif label == 'CONTROL':
            X_Test_CONTROL.append(image)
            Y_Test_CONTROL.append(label)
        
        
print("Image loaded.")
        
#converted generated list into array
X_train = np.array(X_train_img)
Y_train = np.array(Y_train_img)

print("List to array conversion done")

In [ ]:
print(X_train.shape)
X_train, Y_train = shuffle(X_train, Y_train, random_state=42)

#After shuffling sample size remains same
print(X_train.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [ ]:
#Converting String Label to categorical
y_train_new = []
y_test_new = []

for i in Y_train: 
    #Converting String Label to integer i.e # CONTROL ---> 0, AD---> 1, PD ---> 2
    y_train_new.append(labels.index(i))

#Converts a class vector (integers) to binary class matrix
Y_train = to_categorical(y_train_new) 

for i in Y_test:
    y_test_new.append(labels.index(i))

Y_test = to_categorical(y_test_new)

In [ ]:
#check y train
Y_train

In [ ]:
#save the model
def save_model(p_Model, modelName):    
    with open(SAVE_DIR + modelName + '.h5','a') as f:
        print(p_Model, file=f)

In [ ]:
print(str(datetime.now()))
def get_model_bfr_regularization(model_name):
    # Initiate EfficientNetB0 model
    efficientnet_B0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X, image_size_Y, 3))
    print("If download not started, means already downloaded")
    
    """
    for layer in efficientnet_B0.layers:
        layer.trainable = False
    """
    
    # Output layer of the model
    model = efficientnet_B0.output
    
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dropout(0.5)(model)    
    model = tf.keras.layers.Dense(3, activation='softmax')(model)

    #Merge input and Output of model
    model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

    #model.name = 'model_summary'
    
    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
#efficientnetB0 with augmentation and tuning defination

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

print(str(datetime.now()))
def get_model_aftr_augmentation(model_name):
    # Initiate EfficientNetB0 model
    inputs = layers.Input(shape=(image_size_X, image_size_Y, 3))
    x = img_augmentation(inputs)
    efficientnet_B0 = EfficientNetB0(include_top=False, input_tensor=x, weights='imagenet')
    print("If download not started, means already downloaded")
    
    """
    for layer in efficientnet_B0.layers:
        layer.trainable = False
    """
    
    # Output layer of the model
    model = efficientnet_B0.output
    
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dropout(0.5)(model)    
    model = tf.keras.layers.Dense(3, activation='softmax')(model)

    #Merge input and Output of model
    model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

    #model.name = 'model_summary'
    
    model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
from keras.layers import Dense, Input, Activation, add, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model

def get_model_aftr_aug_tuning(model_name):
    
    print("get_model_aftr_aug_tuning")
    # Initiate EfficientNetB0 model
    inputs = layers.Input(shape=(image_size_X, image_size_Y, 3))
    x = img_augmentation(inputs)
    model_ENB0 = EfficientNetB0(include_top=False, input_tensor=x, weights='imagenet')
    print("If download not started, means already downloaded")
    
    #model_ENB0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X ,image_size_X, 3))
    model_ENB0.trainable = False
    #model_ENB0.summary()
    
    model = Sequential()
    model.add(model_ENB0)
    
    model.summary()
    
    # Unfreezing
    model_ENB0.trainable = True
    set_trainable = False

    for layer in model_ENB0.layers:
        if layer.name == 'block6d_se_excite':
            set_trainable = True
        if set_trainable:
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True
            else:
                layer.trainable = False
        else:
            layer.trainable = False
            
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(3,activation="softmax"))
    model.summary()
    
    return model

In [ ]:
print(str(datetime.now()))
def get_model_after_layering(model_name):
    # Initiate EfficientNetB0 model
    efficientnet_B0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X, image_size_Y, 3))
    print("If download not started, means already downloaded")
    
    """
    for layer in efficientnet_B0.layers:
        layer.trainable = False
    """
    
    # Output layer of the model
    model = efficientnet_B0.output
    
    
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.Dropout(0.25)(model)
    model = tf.keras.layers.Dense(256, activation='relu')(model)
    model = tf.keras.layers.Dropout(0.25)(model)
    model = tf.keras.layers.Dense(32, activation='relu')(model)
    model = tf.keras.layers.Dropout(0.25)(model)
    model = tf.keras.layers.Dense(3, activation='softmax')(model)

    #Merge input and Output of model
    model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

    #model.name = 'model_summary'
    
    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
print(str(datetime.now()))
def get_model_after_layering2(model_name):
    # Initiate EfficientNetB0 model
    efficientnet_B0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X, image_size_Y, 3))
    print("If download not started, means already downloaded")

    # Output layer of the model
    #base_model = efficientnet_B0.output
    
    model= Sequential()
    model.add(efficientnet_B0) 
    model.add(tf.keras.layers.Flatten()) 

    #Model summary
    model.summary()
    
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1024,activation=('relu'),input_dim=512))

    model.add(tf.keras.layers.Dense(512,activation=('relu'))) 
    model.add(tf.keras.layers.Dense(256,activation=('relu'))) 
    model.add(tf.keras.layers.Dropout(.1))
    model.add(tf.keras.layers.Dense(128,activation=('relu')))
    model.add(tf.keras.layers.Dropout(.1))
    model.add(tf.keras.layers.Dense(64,activation=('relu')))
    model.add(tf.keras.layers.Dropout(.1))
    model.add(tf.keras.layers.Dense(3,activation=('softmax'))) 

    #Checking the final model summary
    model.summary()

    #Merge input and Output of model
    #model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

    #model.name = 'model_summary'
    
    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
print(str(datetime.now()))
def get_model_after_layering3(model_name):
    print('get_model_after_layering3')
    # Initiate EfficientNetB0 model
    efficientnet_B0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X, image_size_Y, 3))
    print("If download not started, means already downloaded")

    # Output layer of the model
    #base_model = efficientnet_B0.output
    
    model=Sequential()
    chanDim=-1
    model.add(efficientnet_B0)
    model.add(GlobalAveragePooling2D())
    #model.add(Dense(1024,activation=('relu')))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.5))
    model.add(Dense(512,activation=('relu')))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation=('relu')))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.3))
    model.add(Dense(128,activation=('relu')))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.2))
    model.add(Dense(3,activation=('softmax')))
    
    #Checking the final model summary
    model.summary()

    #Merge input and Output of model
    #model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

    #model.name = 'model_summary'
    
    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
#efficientnetB7 with augmentation and tuning defination
#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

print(str(datetime.now()))
def get_model_efficientnetB7(model_name):
    # Initiate EfficientNetB0 model
    inputs = layers.Input(shape=(image_size_X, image_size_Y, 3))
    x = img_augmentation(inputs)
    efficientnet_B7 = EfficientNetB7(include_top=False, input_tensor=x, weights='imagenet')
    print("If download not started, means already downloaded")
    
    """
    for layer in efficientnet_B0.layers:
        layer.trainable = False
    """
    
    # Output layer of the model
    model = efficientnet_B7.output
    
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dropout(0.2)(model)    
    model = tf.keras.layers.Dense(3, activation='softmax')(model)

    #Merge input and Output of model
    model = tf.keras.models.Model(inputs=efficientnet_B7.input, outputs=model)

    #model.name = 'model_summary'
    
    model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
print(str(datetime.now()))
def get_model_aftr_regularization(model_name):
    # Initiate EfficientNetB0 model
    efficientnet_B0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X, image_size_Y, 3))
    print("If download not started, means already downloaded")

    # Output layer of the model
    model = efficientnet_B0.output
    
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dropout(0.5)(model)    
    model = tf.keras.layers.Dense(64, kernel_regularizer=regularizers.l2(0.01), activation='softmax')(model)
    model = tf.keras.layers.Dense(3, activation='softmax')(model)

    #Merge input and Output of model
    model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)
    
    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    
    print("Model is ready")
    
    return model

In [ ]:
def get_model_aftr_tuning(model_name):
    model_ENB0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size_X ,image_size_X, 3))
    model_ENB0.trainable = False
    model_ENB0.summary()
    
    model = Sequential()
    model.add(model_ENB0)
    
    
    # Unfreezing
    model_ENB0.trainable = True
    set_trainable = False

    for layer in model_ENB0.layers:
        if layer.name == 'block6d_se_excite':
            set_trainable = True
        if set_trainable:
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True
            else:
                layer.trainable = False
        else:
            layer.trainable = False
            
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(3,activation="softmax"))
    #model.summary()
    
    return model

In [ ]:
def get_model_VGG19(model_name):
    base_model_VGG19 = tf.keras.applications.VGG19(input_shape=(image_size_X, image_size_Y,3),
                                               include_top=False,
                                               weights = "imagenet"
                                            )
    base_model_VGG19.trainable = False
    
    model_vgg = tf.keras.Sequential
    ([
        base_model_VGG19,
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2, activation="softmax")
    ])
    
    model_vgg.summary()
    return model_vgg

In [ ]:
print(str(datetime.now()))
#mesurments metrix for the training of the model
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc')]

#custom_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

print(custom_optimizer)

In [ ]:
print(str(datetime.now()))

#tensorBoard = TensorBoard(log_dir = SAVE_DIR + "logs")
tensorBoard = TensorBoard(log_dir = SAVE_DIR)

reduce_lr = ReduceLROnPlateau(
    #monitor='val_loss',
    monitor='val_acc',
    factor=0.3,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.001
)

#patience = epochs_size means no early stopping
es = EarlyStopping(
    #monitor='val_loss',
    monitor='val_acc',
    patience=epochs_size,
    verbose=1,
    mode='auto',
    restore_best_weights=True
) 


def define_checkPoint(model_name):
        
    checkPoint = ModelCheckpoint(
        SAVE_DIR + model_name + '.tf',
        #monitor='val_loss',
        monitor='val_acc',
        verbose=1,
        save_best_only=True,
        mode='auto'
    )
    
    return checkPoint
    

In [ ]:
import joblib

print(str(datetime.now()))
def print_output(model, history, model_name):
    
    #save the model
    #filename = "completed_model.joblib"
    #joblib.dump(model, SAVE_DIR + filename)
    
    trend_of_the_metrics = model_name + '_' + 'trend_of_the_metrics.png'
    Accuracy_vs_Loss_of_Training = model_name + '_' + 'Accuracy_vs_Loss_of_Training.png'
    Training_vs_Validation_Accuracy = model_name + '_' + 'Training_vs_Validation_Accuracy.png'
    Training_vs_Validation_Loss = model_name + '_' + 'Training_vs_Validation_Loss.png'
    classification_report_name = model_name + '_' + 'classification_report.txt'
    confusion_matrix_name = model_name + '_' + 'confusion_matrix.png'
    model_name_to_save = model_name + '.tf'
        
    print('Plotting the trend of the metrics during training by the model')
    fig, ax = plt.subplots(1, 3, figsize = (60, 10))
    ax = ax.ravel()

    for i, metric in enumerate(["acc", "auc", "loss"]):
        print(i)
        print (metric)
        ax[i].plot(history.history[metric])
        ax[i].plot(history.history["val_" + metric])
        ax[i].set_title("Model {}".format(metric))
        ax[i].set_xlabel("Epochs")
        ax[i].set_ylabel(metric)
        ax[i].legend(["train", "val"])

    #save plot
    plt.savefig(SAVE_DIR + trend_of_the_metrics, bbox_inches = 'tight')
    
    
    print('Plotting training, validation and loss curve')
    get_ac = history.history['acc']
    get_los = history.history['loss']
    val_acc = history.history['val_acc']
    val_loss = history.history['val_loss']

    epochs = range(len(get_ac))

    #Accuracy vs Loss of Training
    plt.plot(epochs, get_ac, 'g', label='Training Accuracy')
    plt.plot(epochs, get_los, 'r', label='Training Loss')
    plt.title('Accuracy vs Loss of Training')
    plt.xlabel('No of Epochs')
    plt.ylabel('Accuracy/Loss of Training')
    plt.legend(loc=0)
    plt.savefig(SAVE_DIR + Accuracy_vs_Loss_of_Training, bbox_inches = 'tight')
    plt.figure()


    #Figure Training vs Validation Accuracy
    plt.plot(epochs, get_ac, 'g', label='Training Data Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Data Accuracy')
    plt.title('Training vs Validation Accuracy of the proposed model')
    plt.xlabel('No of Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc=0)

    plt.savefig(SAVE_DIR + Training_vs_Validation_Accuracy, bbox_inches = 'tight')
    plt.figure()



    #Figure Training vs Validation Loss
    plt.plot(epochs, get_los, 'g', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training vs Validation Loss of the proposed model')
    plt.xlabel('No of Epochs')
    plt.ylabel('Loss')
    plt.legend(loc=0)
    plt.savefig(SAVE_DIR + Training_vs_Validation_Loss, bbox_inches = 'tight')
    plt.figure()
    plt.show()
    
    
    print('classification_report report of training and validation')
    pred = model.predict(X_test)
    pred_labels = np.argmax(pred, axis=1)
    actual_label = np.argmax(Y_test, axis=1)
    
    #print(classification_report(actual_label, pred_labels, target_names=labels))
    report = classification_report(actual_label, pred_labels, target_names=labels)
    print(report)
    report_path = classification_report_name
    text_file = open(report_path, "w")
    n = text_file.write(report)
    text_file.close()
    
        
    
    print('Preparing Confusion matrix')
    cnf = confusion_matrix(actual_label, pred_labels)
    plt.figure(figsize=(8,6), dpi=70, facecolor='w', edgecolor='k')
    ax = sns.heatmap(cnf, cmap='Blues',annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
    plt.title('Alzheimer\'s and Parkinson\'s Classification')
    plt.xlabel('Prediction')
    plt.ylabel('Ground Truth')

    #save plot
    plt.savefig(SAVE_DIR + confusion_matrix_name, bbox_inches = 'tight')
    plt.show(ax)
    
    
    #Load save the model for further use
    #saved_model = tf.keras.models.load_model(SAVE_DIR + model_name_to_save)

    #Show Model Architecture
    #plot_model(saved_model, to_file='efficient_net_B0.png', show_shapes=True, show_layer_names=True)
        

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#Model efficientnetb0
def model_efficientnetb0(model_type):
    print(str(datetime.now()))
    model_name = 'b0-CNN' + '-' + str(model_type)
    #Configures the model for training
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    
    #B0-CNN
    if model_type == 1:
        model = get_model_bfr_regularization(model_name)
        
    elif model_type == 2:
        model = get_model_aftr_regularization(model_name)
    
    elif model_type == 3:
        model = get_model_aftr_augmentation(model_name)
        
    elif model_type == 4:
        model = get_model_aftr_tuning(model_name)
    
    #73%
    elif model_type == 5:
        model = get_model_aftr_aug_tuning(model_name)
        
    elif model_type == 6:
        model = get_model_after_layering(model_name)
        
    elif model_type == 7:
        model = get_model_after_layering2(model_name)
        
    elif model_type == 8:
        model = get_model_after_layering3(model_name)
        
    elif model_type == 50:
        model = get_model_efficientnetB7(model_name) 
        
    else:
        model = get_model_aftr_tuning(model_name)
        
    #model.summary(print_fn=save_model(model, model_name))
    model.summary()
    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)

    # Create an instance of ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    # Fit the data generator to your data
    #datagen.fit(X_train)
    
    from sklearn.model_selection import train_test_split

    print("Data size before splitting (without test data)")
    print(X_train.shape, Y_train.shape)
    
    # Split the data into a training set and a validation set
    X_train_f, X_val, Y_train_f, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)
    
    print("Training data shape")
    print(X_train_f.shape, Y_train_f.shape)

    print("Test data shape")
    print(X_test.shape, Y_test.shape)

    print("Validation data shape")
    print(X_val.shape, Y_val.shape)
    
    #train
    print(str(datetime.now()))
    checkPoint = define_checkPoint(model_name)
    
    if with_augmentation == 1:
        print('Fit the model with data augmentation')
        
        augmented_dataset = datagen.flow(X_train_f, Y_train_f, batch_size=32)
        print('Training Data size after augmentation')
        
        # Get the shape of augmented dataset
        #shows error "AttributeError: 'tuple' object has no attribute 'shape'"
        #print("Shape of x_augmented:", augmented_dataset[0].shape)
        #print("Shape of y_augmented:", augmented_dataset[1].shape)  
        
        #print augmented image
        #print_augmented_images()

        history = model.fit(
                    augmented_dataset,
                    batch_size=batch_size_here,
                    #validation_split=0.1,
                    #validation_data=datagen.flow(X_val, Y_val, batch_size=32),
                    validation_data=(X_val, Y_val),
                    epochs=epochs_size,
                    verbose=1,
                    callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                )
    else:
        print('Fit the model without data augmentation')
        history = model.fit(X_train_f, Y_train_f,
                    batch_size=batch_size_here,
                    #validation_split=0.1,
                    validation_data=(X_val, Y_val),
                    epochs=epochs_size,
                    verbose=1,
                    callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                )
        


    #Evaluating the hybrid model on training, test and validation data

    #train_scores = hybrid_model.evaluate(train_data, train_labels)
    #val_scores = hybrid_model.evaluate(val_data, val_labels)
    test_scores = model.evaluate(X_test, Y_test)

    #print("Training Accuracy of the model: %.2f%%"%(train_scores[1] * 100))
    #print("Validation Accuracy of the model: %.2f%%"%(val_scores[1] * 100))
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))


    #Print final output
    print_output(model, history, model_name)
    
    # Display predicted Images
    #predict_mri(X_test, Y_test)  
    
    """
    try:
        # Save the model in the .pb format
        tf.saved_model.save(model, 'my_model')
    except Exception as e:
        print("An error occurred:", e)
    finally:
        print("Model save Done.")
    """
    
    return history

In [ ]:
#print augmented images
def print_augmented_images():
    # Load image and apply data augmentation
    #image = Image.open('/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/test/AD/AD_2562.png')
    image = image_denoise('/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/test/AD/', 'AD_2562.png')
    
    image_array = np.array(image)
    image_array = np.expand_dims(image_array, axis=0)
    
    dg = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    aug_iter = dg.flow(image_array, batch_size=32)

    # Show augmented images
    fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(16,16))
    for i in range(4):
        for j in range(4):
            axs[i][j].imshow(next(aug_iter)[0].astype(np.uint8))
            axs[i][j].axis('off')
    
    plt.savefig(SAVE_DIR + 'augmented_images.png', bbox_inches = 'tight')
    plt.show()
    

try:
    print_augmented_images()
except Exception as e:
    print("An error occurred:", e)
finally:
    print("Augmented image printing Done.")


In [ ]:
#Not working
from skimage import io
#print augmented images
def print_augmented_images2():
    # Load image and apply data augmentation
    #image = Image.open('/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/test/AD/AD_2562.png')
    #image = image_denoise('/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/test/AD/', 'AD_2562.png')
    
    # Load the image
    img_path = '/kaggle/input/working-ds/AD_PD_DETECTION/ORIGINAL_AD_PD_DATA/test/AD/AD_2562.png'
    img = io.imread(img_path)

    # Reshape the image to (1, height, width, channels) for use with ImageDataGenerator
    img = img.reshape((1,) + img.shape)

    # Create an instance of the ImageDataGenerator class with your set of augmentation options
    data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
    )

    # Apply the augmentation to the image
    batch_size = 1
    img_generator = data_gen.flow(img, batch_size=batch_size)
    aug_imgs, _ = next(img_generator)

    # Plot the augmented image
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(aug_imgs[0].astype('uint8'))
    plt.show()
    
"""
try:
    print_augmented_images2()
except Exception as e:
    print("An error occurred:", e)
finally:
    print("Printing Done.")
"""

In [ ]:
#model VGG19
def model_VGG19_notworking():
    print(str(datetime.now()))
    model_name = 'hybrid_VGG19'
    #Configures the model for training
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    #model = get_model_bfr_regularization()
    #model = get_model_VGG19(model_name)

    base_model_VGG19 = tf.keras.applications.VGG19(input_shape=(image_size_X, image_size_Y,3),
                                                   include_top=False,
                                                   weights = "imagenet"
                                                )
    base_model_VGG19.trainable = False

    model = tf.keras.Sequential
    ([
        base_model_VGG19,
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3, activation="softmax")
    ])

    #model.summary(print_fn=save_model(model, model_name))
    #model.summary()
    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)

    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    #model.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(lr = 0.001) , metrics='accuracy')

    #train
    print(str(datetime.now()))
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

In [ ]:
#VGG16
def model_VGG16():
    print(str(datetime.now()))
    model_name = 'hybrid_VGG16'
    conv_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

    for layer in conv_model.layers:
        layer.trainable = False

    # flatten the output of the convolutional part: 
    x = tf.keras.layers.Flatten()(conv_model.output)
    # three hidden layers
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    # final softmax layer with three categories 
    predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

    # creating the full model:
    model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)
    #model.summary()
    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)

    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    #model.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(lr = 0.001) , metrics='accuracy')

    #train
    print(str(datetime.now()))
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

In [ ]:
#VGG19
def model_VGG19():
    print(str(datetime.now()))
    model_name = 'hybrid_VGG19'
    vgg19 = tf.keras.applications.vgg19

    conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in conv_model.layers: 
        layer.trainable = False

    x = tf.keras.layers.Flatten()(conv_model.output)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    predictions = tf.keras.layers.Dense(3, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)
    model.summary()

    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)

    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    #model.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(lr = 0.001) , metrics='accuracy')

    #train
    print(str(datetime.now()))
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

In [ ]:
#ResNet50
def model_ResNet50():
    print(str(datetime.now()))
    model_name = 'hybrid_ResNet50'
    resnet50 = tf.keras.applications.resnet50

    conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    
    for layer in conv_model.layers:
        layer.trainable = False
    
    x = tf.keras.layers.Flatten()(conv_model.output)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    x = tf.keras.layers.Dense(100, activation='relu')(x)
    predictions = tf.keras.layers.Dense(3, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)
    model.summary()

    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)

    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    #model.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(lr = 0.001) , metrics='accuracy')

    #train
    print(str(datetime.now()))
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

In [ ]:
#efficientnetB0 with augmentation and tuning defination
#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def get_EfficientNetB0_with_aug(model_name):
    inputs = layers.Input(shape=(image_size_X, image_size_Y, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    #model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(3, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=model_name)
    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)
    
    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    
    return model

def get_EfficientNetB0_with_aug_tune(model_name):
    inputs = layers.Input(shape=(image_size_X, image_size_Y, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False
    
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(3, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=model_name)
    with open(SAVE_DIR + model_name + '.h5','a') as f:
        print(model, file=f)
        
    model.compile(optimizer=custom_optimizer, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)
    
    return model


def model_EfficientNetB0_aug():
    print(str(datetime.now()))
    model_name = 'EfficientNetB0_aug'
    
    model = get_EfficientNetB0_with_aug(model_name)
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

def model_EfficientNetB0_aug_tune():
    print(str(datetime.now()))
    model_name = 'EfficientNetB0_aug_tune'
    
    model = get_EfficientNetB0_with_aug_tune(model_name)
    checkPoint = define_checkPoint(model_name)
    history = model.fit(X_train, 
                        Y_train,
                        batch_size=batch_size_here,
                        validation_split=0.1,
                        epochs=epochs_size,
                        verbose=1,
                        callbacks=[tensorBoard, checkPoint, reduce_lr, es]
                       )

    #Evaluating the hybrid model on test data
    test_scores = model.evaluate(X_test, Y_test)
    print(model_name + ": Testing Accuracy of the model: %.2f%%"%(test_scores[1] * 100))

    #Print final output
    print_output(model, history, model_name)

In [ ]:
#run below 80%
#model_EfficientNetB0_aug_tune()

In [ ]:
#run model
with_augmentation = 1
history = model_efficientnetb0(8)


In [ ]:
from random import randint
# define funtion to predict and Display Test Images result by the model
def predict_mri(images, labels, ready_model):
    
    #image class names
    class_names =dict(zip([0,1,2,3], CLASSES))
    print(class_names)
    
    plt.figure(figsize=(20, 20))
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        indx = randint(0, 700)
        plt.imshow(images[i])
        
        predictions = ready_model.predict(tf.expand_dims(images[indx], 0))
        score = tf.nn.softmax(predictions[0])
        
        if(class_names[np.argmax(labels[indx])]==class_names[np.argmax(score)]):
            plt.title("Actual: "+ class_names[np.argmax(labels[indx])])
            plt.ylabel("Predicted: "+class_names[np.argmax(score)],fontdict={'color':'green'})

        else:
            plt.title("Actual: " + class_names[np.argmax(labels[indx])])
            plt.ylabel("Predicted: " + class_names[np.argmax(score)],fontdict={'color':'red'})

        plt.gca().axes.yaxis.set_ticklabels([])        
        plt.gca().axes.xaxis.set_ticklabels([])
        
    #save plot
    plt.savefig(SAVE_DIR + "prediction_result.png", bbox_inches = 'tight')
    


try:    
    #load saved model
    saved_model = tf.keras.models.load_model('/kaggle/working/b0-CNN-8.tf')
    # Display predicted Images
    predict_mri(X_test, Y_test, saved_model)
except Exception as e:
    print("An error occurred:", e)
finally:
    print("Display predicted Images Done.")

In [ ]:
#print_augmented_images()

In [ ]:
#run model
#model_efficientnetb0(7)

In [ ]:
#run model (97.70 with denoise + tune)
#Tuned
#model_efficientnetb0(4)

In [ ]:
#run model
#model_efficientnetb0(5)

In [ ]:
#run model
##model_EfficientNetB0_aug()

In [ ]:
#run model
#model_EfficientNetB0_aug_tune()

In [ ]:
#run model
#model_VGG16()

In [ ]:
#run model
#model_VGG19()

In [ ]:
#run model
#model_ResNet50()

In [ ]:
#https://www.kaggle.com/general/65351

import os
os.chdir(r'/kaggle/working')

!tar -czf Landscapes.tar.gz b0-CNN-8.tf

from IPython.display import FileLink

FileLink(r'Landscapes.tar.gz')

In [ ]:
#Uploading Image and Showing its Class
def imagePrediction(upload):
    for name, fileinfo  in uploader.value.items():
        image = Image.open(io.BytesIO(fileinfo['content']))
        
    images = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    images = cv2.resize(images,(150, 150))
    images = images.reshape(1, 150, 150, 3)
    prd = model.predict(images)
    prd = np.argmax(prd, axis = 1)[0]
    
    
    if prd == 0:
        prd = "CONTROL"
    elif prd == 1:
        prd = "AD"
    elif prd ==2:
        prd = "PD"
        
    print(f'Model Predict That is  a {prd}')

In [ ]:
uploader = widgets.FileUpload()
display(uploader)

In [ ]:
##### button = widgets.Button(description= "Predict")
"""
out = widgets.Output()

def on_button_click(_):
    with out:
        clear_output()
        try:
            imagePrediction(uploader)
        except:
            print("Please Enter the Correct Image files")
            
            
button.on_click(on_button_click)
widgets.VBox([button, out])
"""